In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv

print('Importing packages (done)')

Importing packages (done)


In [2]:
# Login to Linkedin

# Open Chrome and Access Linkedin login site
driver = webdriver.Chrome("C:/Users/guedri/Downloads/chromedriver_win32/chromedriver.exe")
sleep(2)
url = 'https://www.linkedin.com/login'
driver.get(url)
print('Initializing a driver (done)')
sleep(2)

# Import username and password
credential = open('credentials.txt')
line = credential.readlines()
username = line[0]
password = line[1]
print('Importing the login credentials (done)')
sleep(2)

# Input the login credentials
email_field = driver.find_element_by_id('username')
email_field.send_keys(username)
print('Input email (done)')
sleep(3)

password_field = driver.find_element_by_name('session_password')
password_field.send_keys(password)
print('Input password (done)')
sleep(2)

# Click the Login button
signin_field = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
signin_field.click()
sleep(3)

print('Login to Linkedin (done)')

Initializing a driver (done)
Importing the login credentials (done)
Input email (done)
Input password (done)
Login to Linkedin (done)


In [3]:
# Search for the profiles we want to scrape

# Locate the search bar element
search_field = driver.find_element_by_xpath('//*[@class="search-global-typeahead__input always-show-placeholder"]')

# Input the search query to the search bar
search_query = input('What profile do you want to scrape? ')
search_field.send_keys(search_query)

# Search
search_field.send_keys(Keys.RETURN)

print('Search for profiles (done)')

What profile do you want to scrape? software engineer
Search for profiles (done)


In [4]:
# Scrape the URLs of the profiles

# A function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'app-aware-link')
    all_profile_URL = []
    for profile in profiles:
        profile_URL = profile.get('href')
        if "https://www.linkedin.com/in/" in profile_URL:
            if profile_URL not in all_profile_URL:
                all_profile_URL.append(profile_URL)
    return all_profile_URL

# Navigate through many pages, and extract the profile URLs of each page

input_page = int(input('How many pages you want to scrape: '))
URLs_all_page = []

if(input_page > 1):
    for page in range(input_page):
        URLs_one_page = GetURL()
        sleep(2)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
        sleep(3)
        next_button = driver.find_element_by_class_name('artdeco-pagination__button--next')
        driver.execute_script("arguments[0].click();", next_button)
        URLs_all_page = URLs_all_page + URLs_one_page
        sleep(2)
else:
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)
print(URLs_all_page)
print('Scrape the URLs (done)')

How many pages you want to scrape: 3
['https://www.linkedin.com/in/chaima-amri-20b1b9151', 'https://www.linkedin.com/in/ACoAADQeB3UB_dszV3t0BDY_BnYm8wW1ExWhiBU', 'https://www.linkedin.com/in/ACoAADJ76vYBzMcQQvdQ-lDm-jZyOP7A_IMNXfE', 'https://www.linkedin.com/in/zayneb-hammami', 'https://www.linkedin.com/in/ACoAADAMf40BAw04GvLSs_ED9mHofIVTFfvSBDM', 'https://www.linkedin.com/in/medwaeldraoui', 'https://www.linkedin.com/in/ACoAAC2k4kYB0XlrePfBruWacEp6eriYe5GI7OA', 'https://www.linkedin.com/in/ACoAAC15jewBPkOckIWhUQ_vlodeDt0OzOpcQEY', 'https://www.linkedin.com/in/ACoAAClW_KQBlX-4rr6YawaQM7qrfmZ0KipyvAk', 'https://www.linkedin.com/in/ACoAAAoFpeEBFXD6V1iLOwGuuZdXtroWv6-Xc1g', 'https://www.linkedin.com/in/shiraz-hasni-96489613a', 'https://www.linkedin.com/in/ACoAAC588coBQXtoRn_aa2PQs6hEIH26hOvtLl0', 'https://www.linkedin.com/in/ACoAAC5WRP4BETh1Ye64ILDqQwIq1GfAtfB3ev8', 'https://www.linkedin.com/in/rzaigui-amani-2252a2140', 'https://www.linkedin.com/in/ACoAACRrtD0B4o_KTdGhIvW09AUw41q1KZ3_u7c',

In [5]:
# Scrape the data of 1 Linkedin profile, and write the data to a .CSV file

with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('Accessing profile: ', linkedin_URL)

        page_source = BeautifulSoup(driver.page_source, "html.parser")

        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        
        name = info_div.find('li', class_='inline t-24 t-black t-normal break-words').get_text().strip() #Remove unnecessary characters 
        print('--- Profile name is: ', name)
        location = info_div.find('li', class_='t-16 t-black t-normal inline-block').get_text().strip() #Remove unnecessary characters 
        print('--- Profile location is: ', location)
        title = info_div.find('h2', class_='mt1 t-18 t-black t-normal break-words').get_text().strip()
        print('--- Profile title is: ', title)
        writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
        print('\n')

        """
        info_loc = info_div.find_all('ul')
        name = info_loc[0].find('li').get_text().strip() #Remove unnecessary characters 
        print('--- Profile name is: ', name)
        location = info_loc[1].find('li').get_text().strip() #Remove unnecessary characters 
        print('--- Profile location is: ', location)
        title = info_div.find('h2').get_text().strip()
        print('--- Profile title is: ', title)
        writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
        print('\n')
        """

print('Mission Completed!')

Accessing profile:  https://www.linkedin.com/in/chaima-amri-20b1b9151


AttributeError: 'NoneType' object has no attribute 'find'